## affine gap penalty string distance

> input: food names

> output: words to remove / synomyms

In [1]:
# python version
import sys
print "system_info: %s"%sys.version
# current working directory
import os
print "path_info: %s"%os.getcwd() 

system_info: 2.7.14 |Anaconda custom (64-bit)| (default, Oct 27 2017, 18:21:12) 
[GCC 7.2.0]
path_info: /home/yueliu/Desktop/workspace_yue/Documentation_201712


In [2]:
import myfunctions as f
f.today()

'20180308'

In [3]:
## output from food synset entities
file_name = "food_entities__20180308.p"
df_retrived = f.retrive_file(file_name)

retrived: food_entities__20180308.p


In [4]:
print df_retrived.shape
df_retrived.head(2)

(92298, 12)


,index,food_name,locs,synonyms,menu_names,restaurant,review,review.clean_text,review.restaurant,review.text,review.image,short_name
0,food_0,-PRON- ann,"[deliveroo/food/68155, deliveroo/food/AV062oM7...",[em ann],"[(Em Ann M., 3)]","[delivery_3748, delivery_3749, delivery_3750]",[burppleinitial/review/vGui2G8q],[maggie mee goreng from the rooftop at screeni...,"[(burpple_12581, 1)]",[Maggi Mee Goreng from The Rooftop at Screenin...,[vGui2G8q],-PRON- ann
1,food_1,-PRON- bai vege,"[foodpanda/menu_item/AV44Ejd8W4PLiMnQOaiu, foo...",[you bai vege],"[(H6. You Bai Vege, 2)]","[delivery_221, delivery_222]",NaN,NaN,NaN,NaN,NaN,-PRON- bai vege


In [5]:
food_names = df_retrived["food_name"].tolist()

In [ ]:
import pandas as pd 
try:
    file_name = "drink_20180308.csv"
    drink_lst = pd.read_csv(file_name, header=None)[0].tolist()
except:
    drinl_lst = []
drink_lst = [""] + drink_lst

In [ ]:
food_names = [s for s in sorted(list(set(food_names))) if s not in drink_lst]
print len(food_names)

In [ ]:
# count frequency of occurance of the unigram tokens
tokens = [s.split() for s in food_names]
from collections import Counter
c = Counter([item for sublist in tokens for item in sublist])
count_tokens = sorted(c.items(), key=lambda x: -x[1])
count_tokens[:5]

In [ ]:
# all tokens
unique_tokens = [s[0] for s in count_tokens]
print "number of uinique tokens: %d"%len(unique_tokens)

# Use affine gap penalty string distance

In [ ]:
# example 
import affinegap
d1 = affinegap.affineGapDistance('foo', 'bar')
d2 = affinegap.affineGapDistance('foo', 'bar',
                                 matchWeight = 1,
                                 mismatchWeight = 11,
                                 gapWeight = 10,
                                 spaceWeight = 7,
                                 abbreviation_scale = .125)
d3 = affinegap.normalizedAffineGapDistance('foo', 'bar')

print d1, d2, d3

In [ ]:
sample_names = food_names[140:150]
sample_names 

In [ ]:
sample_dist = []
sample_dist_norm = []
import itertools
for s0, s1 in itertools.combinations(sample_names,2):
    sample_dist.append([affinegap.affineGapDistance(s0, s1), s0, s1])
    sample_dist_norm.append([affinegap.normalizedAffineGapDistance(s0, s1), s0, s1]) 
sample_dist = sorted(sample_dist, key=lambda x: x[0])
sample_dist_norm = sorted(sample_dist_norm, key=lambda x: x[0])

In [ ]:
sample_dist[:5]

In [ ]:
sample_dist_norm[:5]

In [ ]:
%%time
dist_norm = []
import itertools
for s0, s1 in itertools.combinations(food_names[1000:2000],2):
    try:
        d = affinegap.normalizedAffineGapDistance(s0, s1)
    except: 
        d = -1
    dist_norm.append([d, s0, s1])

In [ ]:
%%time
dist_norm = []
import itertools
for s0, s1 in itertools.combinations(food_names,2):
    try:
        d = affinegap.normalizedAffineGapDistance(s0, s1)
    except: 
        d = -1
    dist_norm.append([d, s0, s1])

In [ ]:
dist_norm = sorted(dist_norm, key=lambda x: x[0])
print "dist pairs: %d"%len(dist_norm)
threshold = 1
res = [l for l in dist_norm if (l[0] <threshold and l[0]>0.5)]
print "dist smaller than threshold=%d: %d"%(threshold, len(res))

In [ ]:
# save the file back to words to remove
import myfunctions as f
file_name = "dist_norm_" + f.today() + ".csv"
import csv
with open(file_name, "wb") as output:
    writer = csv.writer(output)
    writer.writerows(res) 
print ("saved: %s" % file_name)

>  Example pair 

> 0.559782624,	aloo mutter sukhewali,	aloo mutter sukhewali ali

    consider multi-filed matching
    -> description similarity > some desc_sim threshold
    -> offered by the same restaurants 
